### This notebook is an experiment if a single LSTM neuron can model exponential moving average, average of variance, bollinger bands


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
%matplotlib inline
from keras.layers import Input, SimpleRNN
from keras.models import Model


T = 1000
# create random walk w/ drift and it's moving average
a = np.cumsum(np.array([random.random() for _ in range(T)])) - np.linspace(0, T/2, T)
data = pd.DataFrame({'a': a})
data['ma'] = np.concatenate([[0], pd.ewma(data.a, com=50)[:-1].values])# so mu_t = 15/16*mu(t-1) + 1/16*x_t
data.plot()
x = data.a.values.reshape((1,T,1))
y = data.ma.values.reshape((1,T,1))

# unrolled
inputs = Input(shape=(T,1))
MA_contender = SimpleRNN(1, activation='linear', return_sequences = True, unroll = True)(inputs)
model = Model(inputs=inputs, outputs=MA_contender)
model.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mae'])
model.fit(x, y, epochs = 20, verbose = 0)
data['y_hat'] = model.predict(x).reshape((T))
data.plot()

# rolled
inputs = Input(shape=(T,1))
MA_contender = SimpleRNN(1, activation='linear', return_sequences = True, unroll = False)(inputs)
model = Model(inputs=inputs, outputs=MA_contender)
model.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mae'])
model.fit(x, y, epochs = 50, verbose = 0)
data['y_hat'] = model.predict(x).reshape((T))
data.plot()

/home/hristo/mlenv3/lib/python3.5/site-packages/ipykernel_launcher.py:14: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(adjust=True,ignore_na=False,com=50,min_periods=0).mean()
  
